In [88]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report
from keras.models import Sequential
from keras.layers import Dense
import keras_tuner as kt
from tensorflow import keras

In [89]:
#loading Dataset
churn=pd.read_csv('E:\projects\Churn_Prediction_ANN\Churn_Predictions.csv')

In [90]:
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [91]:
churn.shape

(10000, 14)

In [92]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [93]:
#dropping feature beacause of high number of distinct values.
churn.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [94]:
churn.shape

(10000, 11)

In [95]:
cat=churn[['Geography','Gender']]

In [96]:
cont=churn.drop(cat.columns,axis=1)

In [97]:
cat.shape

(10000, 2)

In [98]:
cont.shape

(10000, 9)

In [99]:
#Handling categorical data
#One-hot encoding
cat_dummies=pd.get_dummies(data=cat)

In [100]:
churn['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [101]:
cat_dummies.head()

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,0,0,1,0
1,0,0,1,1,0
2,1,0,0,1,0
3,1,0,0,1,0
4,0,0,1,1,0


In [102]:
churn_final=pd.concat([cat_dummies,cont],axis=1)

In [103]:
churn_final.head()

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,0,0,1,0,619,42,2,0.00,1,1,1,101348.88,1
1,0,0,1,1,0,608,41,1,83807.86,1,0,1,112542.58,0
2,1,0,0,1,0,502,42,8,159660.80,3,1,0,113931.57,1
3,1,0,0,1,0,699,39,1,0.00,2,0,0,93826.63,0
4,0,0,1,1,0,850,43,2,125510.82,1,1,1,79084.10,0


In [104]:
#check for missings
churn_final.isnull().sum()

Geography_France     0
Geography_Germany    0
Geography_Spain      0
Gender_Female        0
Gender_Male          0
CreditScore          0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [105]:
#spliting X and Y features
X=churn_final.drop(columns=['Exited'])
Y=churn_final['Exited'].copy()

In [106]:
print(X.shape,Y.shape)

(10000, 13) (10000,)


In [107]:
X_stan=StandardScaler()

In [108]:
X_stan=X_stan.fit_transform(X)

In [109]:
X_stan=pd.DataFrame(X_stan,columns=X.columns)

In [110]:
X_stan

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0.997204,-0.578736,-0.573809,1.095988,-1.095988,-0.326221,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886
1,-1.002804,-0.578736,1.742740,1.095988,-1.095988,-0.440036,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534
2,0.997204,-0.578736,-0.573809,1.095988,-1.095988,-1.536794,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687
3,0.997204,-0.578736,-0.573809,1.095988,-1.095988,0.501521,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918
4,-1.002804,-0.578736,1.742740,1.095988,-1.095988,2.063884,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.997204,-0.578736,-0.573809,-0.912419,0.912419,1.246488,0.007457,-0.004426,-1.225848,0.807737,0.646092,-1.030670,-0.066419
9996,0.997204,-0.578736,-0.573809,-0.912419,0.912419,-1.391939,-0.373958,1.724464,-0.306379,-0.911583,0.646092,0.970243,0.027988
9997,0.997204,-0.578736,-0.573809,1.095988,-1.095988,0.604988,-0.278604,0.687130,-1.225848,-0.911583,-1.547768,0.970243,-1.008643
9998,-1.002804,1.727904,-0.573809,-0.912419,0.912419,1.256835,0.293517,-0.695982,-0.022608,0.807737,0.646092,-1.030670,-0.125231


In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=5)

In [112]:
X_train.shape

(8000, 13)

In [113]:
X_test.shape

(2000, 13)

In [114]:
model = Sequential([
    keras.layers.Dense(26, input_shape=(13,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 889us/step - loss: 545.1890 - accuracy: 0.6747
Epoch 2/100
250/250 [==============================] - 0s 892us/step - loss: 64.0207 - accuracy: 0.6811
Epoch 3/100
250/250 [==============================] - 0s 902us/step - loss: 58.4846 - accuracy: 0.6886
Epoch 4/100
250/250 [==============================] - 0s 903us/step - loss: 59.2384 - accuracy: 0.6865
Epoch 5/100
250/250 [==============================] - 0s 905us/step - loss: 47.3815 - accuracy: 0.6789
Epoch 6/100
250/250 [==============================] - 0s 903us/step - loss: 58.6537 - accuracy: 0.6846
Epoch 7/100
250/250 [==============================] - 0s 891us/step - loss: 41.7583 - accuracy: 0.6850
Epoch 8/100
250/250 [==============================] - 0s 921us/step - loss: 50.0249 - accuracy: 0.6852
Epoch 9/100
250/250 [==============================] - 0s 893us/step - loss: 53.9202 - accuracy: 0.6710
Epoch 10/100
250/250 [==============================] - 0s 892u

In [115]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 727us/step - loss: 13.0771 - accuracy: 0.7940


[13.077105522155762, 0.7940000295639038]

In [116]:

yp = model.predict(X_test)

In [117]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [118]:
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [119]:
y_test[:10]

7054    0
442     0
3954    0
2288    0
3196    0
6178    0
8351    0
5658    1
2065    0
413     1
Name: Exited, dtype: int64

In [120]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.99      0.88      1595
           1       0.23      0.01      0.01       405

    accuracy                           0.79      2000
   macro avg       0.51      0.50      0.45      2000
weighted avg       0.68      0.79      0.71      2000

[[1585   10]
 [ 402    3]]


In [121]:
#lets try keras tuner to get optimal number of layers and neurons in each layer.
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [122]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [123]:
tuner.search_space_summary()

Search space summary
Default search space size: 13
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [124]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [132]:
best_model=tuner.get_best_models()[0]

In [136]:
yp_tuner=best_model.predict(X_test)

In [137]:
y_pred_tuner = []
for element in yp_tuner:
    if element > 0.5:
        y_pred_tuner.append(1)
    else:
        y_pred_tuner.append(0)

In [138]:
print(classification_report(y_test,y_pred_tuner))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1595
           1       0.00      0.00      0.00       405

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.44      2000
weighted avg       0.64      0.80      0.71      2000



e:\projects\Churn_Prediction_ANN\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\projects\Churn_Prediction_ANN\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\projects\Churn_Prediction_ANN\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [139]:
#we can clearly see that by tuning our Neural Net gives us better accuracy.